In [3]:
from mido import Message, MidiFile, MidiTrack
import os
from mido import MetaMessage
import random
import mido
import copy
from itertools import chain


# directory name should be midis, at the same level as this file. 

In [4]:
# This function will take in a folder and convert each MIDI file into a list of tokens where each token is a note value of the piece
# It will return a 2D list where each row is a piece and each column represents a token/note of the piece
def tokenize_pieces(midi_folder, track_name):
    midi_pieces = []
    for midi_file_name in os.listdir(midi_folder):
        midsource = MidiFile(midi_folder + "/" + midi_file_name)
        note_token_sequence = []
        for i, track in enumerate(midsource.tracks):
            if track.name == track_name:
                for msg in track:
                    if msg.type == 'note_on':
                         note_token_sequence.append(msg.note)
        midi_pieces.append(note_token_sequence)
    return midi_pieces


In [5]:
# This function will generate n grams given a list of token sequences
def create_n_grams(n, token_sequences):
    
    paddings = n - 1
    start_padding = "<start>"
    end_padding = "<end>"
    n_grams_count = {}

    # Add the start padding (<start>) n-1 number of times
    if paddings > 0:
        for i in range(0, len(token_sequences)):
            for j in range(0, paddings):
                token_sequences[i].insert(0, start_padding)

    # Add the end padding (<end>) once to the end of each sequence
    for i in range(0, len(token_sequences)):
        token_sequences[i].append(end_padding)

    # Iterate through each word in each sequence and using slicing to get the n gram, then add to dictionary/update count
    for sequence in token_sequences:
        for i in range(len(sequence)-n+1): # Source: https://stackoverflow.com/questions/13423919/computing-n-grams-using-python
            gram = sequence[i:i+n] # Source: https://stackoverflow.com/questions/13423919/computing-n-grams-using-python
            gram = tuple(gram)
            if gram in n_grams_count:
                n_grams_count[gram] += 1
            else:
                n_grams_count[gram] = 1

    return n_grams_count


In [6]:
def generate_song_using_n_grams(n_gram_frequencies, k):
    
    # We need to first determine what the "n" is from the input frequency dictionary
    n = len(tuple(n_gram_frequencies.keys())[0])

    # Start with an empty tweet
    song_sequence = []

    # For bigrams and up...
    if n - 1 > 0:
        for i in range(0, n-1):
            song_sequence.insert(0, "<start>")
    
        while song_sequence[len(song_sequence)-1] != "<end>":
            
            # Slice the song so we can get the previous n-1 gram that comes before the predicted token
            previous_token_sequence = song_sequence[len(song_sequence)-(n-1):len(song_sequence)]

            # Using dictionary comprehension, create a dictionary containing all n-grams that contain the previous token sequence
            matched_dictionary = {k:v for k,v in n_gram_frequencies.items() if k[0:n - 1] == tuple(previous_token_sequence)}

            # Prepare a list of possible choices along with their corresponding weights
            choices = []
            choices_weights = []

            # Populate the choices and weights by iterating through the keys in the matched dictionary 
            matched_dictionary_keylist = list(matched_dictionary.keys())
            for j in range(0, len(matched_dictionary_keylist)):
                # insert the last word of the tuple key insert it as a choice
                choices.append(matched_dictionary_keylist[j][len(matched_dictionary_keylist[j])-1])

                # Weights are calculated by taking the actual count of the key and dividing it by all that is found
                choices_weights.append(matched_dictionary[matched_dictionary_keylist[j]] / len(matched_dictionary) + k)
            
            chosen = random.choices(choices, weights=choices_weights, k=1)
            song_sequence.append(chosen[0])
    else:
        # Special case for unigrams only
        choices = []
        choices_weights = []
        n_gram_frequencies_keylist = list(n_gram_frequencies.keys())
        for j in range(0, len(n_gram_frequencies_keylist)):
            # insert the last word of the tuple key as a choice
            choices.append(n_gram_frequencies_keylist[j][0])
            choices_weights.append(n_gram_frequencies[n_gram_frequencies_keylist[j]] / len(n_gram_frequencies) + k)
        
        # Adding the first item so tweet isn't empty
        song_sequence.append(random.choices(choices, weights=choices_weights, k=1)[0])
        while song_sequence[len(song_sequence)-1] != "<end>":
            chosen = random.choices(choices, weights=choices_weights, k=1)
            song_sequence.append(chosen[0])
    
    # Removing the paddings and converting the grams of the tweet into a string
    # return ' '.join([word for word in tweet if word != "<start>" and word != "<end>"])
    for x in range(n-1):
        song_sequence.pop(0)
    song_sequence.pop(-1)
    return song_sequence

In [7]:
def create_midi(right_hand, left_hand):
    mid = MidiFile()
    right_track = MidiTrack()
    left_track = MidiTrack()
    for note in right_hand: 
        right_track.append(mido.Message('note_on', note = note, time = 0))
        right_track.append(mido.Message('note_off', note = note, time = 256))
        
    for note in left_hand: 
        left_track.append(mido.Message('note_on', note = note, time = 0))
        left_track.append(mido.Message('note_off', note = note, time = 256))
    
    mid.tracks.append(right_track)
    mid.tracks.append(left_track)

    mid.save('FULL_PIANO_SONG.mid')
    

In [8]:
def remove_chords(list_of_messages):
    messages_no_chords = []
    
    look_for_end = False
    prev_start_note = -1
    for msg in list_of_messages:
        
        if look_for_end:
            if msg.note == prev_start_note and msg.type == 'note_off':
                messages_no_chords.append(msg)
                look_for_end = False
        
        if msg.type == 'note_on' and not look_for_end:
            messages_no_chords.append(msg)
            prev_start_note = msg.note
            look_for_end = True
    
    return messages_no_chords

In [9]:
def create_list_of_messages(midi_folder, track_name):
    list_of_messages = []
    for midi_file_name in os.listdir(midi_folder):
        midsource = MidiFile(midi_folder + "/" + midi_file_name)
        track_messages = []
        for i, track in enumerate(midsource.tracks):
            if track.name == track_name:
                for msg in track:
                    track_messages.append(msg)
        no_chord_messages = remove_chords(track_messages)
        list_of_messages.append(no_chord_messages)   
    return list_of_messages

In [10]:
def create_times_list(list_of_messages):
    times_list = [0]
    time_sum = 0
    for msg in list_of_messages:
        time = msg.time
        time_sum += time
        times_list.append(time_sum)
    return times_list

In [11]:
def convert_list_of_messages_to_notes(messages_list):
    answer = []
    for i in range(len(messages_list)):
        notes = []
        for j in range(len(messages_list[i])):
            message = messages_list[i][j]
            if message.type == 'note_on':
                notes.append(message.note)
        answer.append(notes)
    return answer
    

In [12]:
# Convert time lists into right and left dictionaries for the notes that match
# Ex: right: {0 : A, 4 : B, 8 : D, etc…..}, left: {0 : C, 4 : E, 8 : F, etc……}

def create_dict(times, notes):
    duration_dict = {time:note for time, note in zip(times,notes)}
    return duration_dict
    


In [13]:
def convert_rhythm_dictionary_to_note_sequences(right_hand_time_dict, left_hand_time_dict):
    
    right_hand = []
    left_hand = []
    
    for time in right_hand_time_dict.keys():
        if time in left_hand_time_dict:
            right_hand.append(right_hand_time_dict[time])
            left_hand.append(left_hand_time_dict[time])
    
    return right_hand, left_hand

In [14]:
# Calculates the transition probabilities given a set of bigrams generated from the above function
def calculate_transition_probabilities(bigrams, possible_states):

    # Given a list of bigram tuples, we need to generate a nested dictionary 
    # where trans_prob[a][b] is the probability of transitioning from state a to state b
    trans_prob = {state1: {state2 : 0 for state2 in possible_states} for state1 in possible_states}

    # Transferring the counts from the bigram dictionary to the trans prob dictionary
    for (start, end) in bigrams:
        trans_prob[start][end] = bigrams[(start, end)]
    
    # Once each transition counts are populated, we divide each count by the total times the starting state occured
    for key in list(trans_prob.keys()):
        trans_prob[key] = {end: count / sum(trans_prob[key].values()) for end, count in trans_prob[key].items()}

    # Removing the <start> probabilities
    trans_prob.pop("<start>")
    return trans_prob


In [15]:
# Calculates the emission probabilities given a list of the sentences and a list of their corresponding POS tags
def calculate_emission_probabilities(sentences_list, pos_list):

    # Given the list of the sentences and their corresponding POS list, we need to make
    # a nested dictionary such that emi_probs[a][x] is the probability of emitting observation x in state a
    emi_probs = {}

    for i in range(0, len(sentences_list)):
        for j in range(0, len(sentences_list[i])):
            word = sentences_list[i][j]
            pos = pos_list[i][j]

            if pos in emi_probs:
                if word in emi_probs[pos]:
                    emi_probs[pos][word] += 1
                else:
                    emi_probs[pos][word] = 1
            else:
                emi_probs[pos] = {}
                emi_probs[pos][word] = 1
    
    # Once we have start_counts populated with the count of each state, we then go through each key and divide the nested
    # dictionary by the count of that key
    for key in list(emi_probs.keys()):
        emi_probs[key] = {observation: count / sum(emi_probs[key].values()) for observation, count in emi_probs[key].items()}

    return emi_probs

In [16]:
def viterbi(observed_sequence, t_probs, e_probs):
    # observed_sequence: a list of observed emissions
    # t_probs: a dictionary where t_probs[a][b] is the probability of transitioning from state a to state b
    # e_probs: a dictionary where e_probs[a][x] is the probability of emitting observation x in state a
    states = list(t_probs.keys())

    # define a 2d list, where the row at index i corresponds to the probabilities of the best paths to the possible states
    trellis = [[0.0 for s in states] for o in observed_sequence]

    # define a 2d list, where each row corresponds to the backpointers to where the best paths came from in the previous step
    backpointers = [[None for s in t_probs] for o in observed_sequence]
    
    # initialize the first row of the trellis
    trellis[0] = [e_probs[s].get(observed_sequence[0], 0) for s in states]

    # iterate through the remaining observations, using the previous row of the trellis to compute the current row
    for t in range(1, len(observed_sequence)):
        for s in range(len(states)):
            # compute the probabilities of coming from each previous state in the trellis row at t-1
            # assign trellis[t][s] to be the probability of the best path to this state and seeing the observation at time t
            # assign backpointers[t][s] to be the index of the previous state you came from
            current_state = states[s]
            prev_times_transitional_max = 0
            prev_state_index = -1

            # Loop through each state of the previous row
            for prev_s in range(len(states)):
                previous_state_name = states[prev_s]
                # Calculate the previous probability times the transitional prob of the current_state given the previous state
                prev_times_transitional = trellis[t-1][prev_s] * t_probs[previous_state_name].get(current_state, 0)
                # If the product is greater than the previous max, update the max as well as the previous state index
                if prev_times_transitional > prev_times_transitional_max:
                    prev_times_transitional_max = prev_times_transitional
                    prev_state_index = prev_s
            
            trellis[t][s] = prev_times_transitional_max * e_probs[current_state].get(observed_sequence[t], 0)
            backpointers[t][s] = prev_state_index

    best_path = []
    
    # Adding the highest probability state of the last row of trellis to best path
    best_path.append(states[trellis[len(trellis)-1].index(max(trellis[len(trellis)-1]))])

    for r in range(len(backpointers) - 1, 0, -1):
        # Get which state the most recent element in best_path is pointing to
        prev_state_pointer = states.index(best_path[0])
        best_path.insert(0, states[backpointers[r][prev_state_pointer]])

    # find the highest probability state in the last row of your trellis
    # iterate backwards through your backpointers, following the previous pointer
    # add the state corresponding to each pointer to best_path
    return best_path

In [17]:
# Getting all the messages of each piece for right and left hand
right_hand_messages = create_list_of_messages("./midis", "Piano right")
left_hand_messages = create_list_of_messages("./midis", "Piano left")

# Populating our time/rhythm sequence for each piece based on the hand
right_hand_times = []
left_hand_times = []
for message in right_hand_messages:
    right_hand_times.append(create_times_list(message))
for message in left_hand_messages:
    left_hand_times.append(create_times_list(message))

# Converting our list of messages into just the note values
right_hand_notes = convert_list_of_messages_to_notes(right_hand_messages)
left_hand_notes = convert_list_of_messages_to_notes(left_hand_messages)

In [18]:
# Setting up our data for the HMM
# Our right hand will be our observed sequence while our left hand is our "hidden state"
right_hand_observations = []
left_hand_hidden_states = []

# Go through each piece, calculate the match dictionary of time:notes for right and left, and convert the result to matching left/right sequences of notes
for i in range(len(right_hand_notes)):
    right_notes = right_hand_notes[i]
    left_notes = left_hand_notes[i]
    right_times = right_hand_times[i]
    left_times = left_hand_times[i]
    
    right_sequence, left_sequence = convert_rhythm_dictionary_to_note_sequences(create_dict(right_times, right_notes), create_dict(left_times, left_notes))
    right_hand_observations.append(right_sequence)
    left_hand_hidden_states.append(left_sequence)

    
# Get a list of all state types + <start> state
all_states = list(set(list(chain.from_iterable(left_hand_hidden_states))))
all_states.append("<start>")   
transition_probabilities = calculate_transition_probabilities(create_n_grams(2, copy.deepcopy(left_hand_hidden_states)), all_states)
emission_probabilities = calculate_emission_probabilities(copy.deepcopy(right_hand_observations), copy.deepcopy(left_hand_hidden_states))

# Generating our right hand using n-grams
right_hand_grams = create_n_grams(4, tokenize_pieces("./midis", "Piano right"))
generated_right_hand = generate_song_using_n_grams(right_hand_grams, 0)

# Generating our left hand using Viterbi's with our generated right hand
generated_left_hand = viterbi(generated_right_hand, transition_probabilities, emission_probabilities)

# Creating a MIDI file using our two hand tracks
create_midi(generated_right_hand, generated_left_hand)


In [19]:

def create_midi():
    mid = MidiFile()
#     right_track = MidiTrack()
    left_track = MidiTrack()
    left_track.append(mido.Message('note_on', note = 81, time = 0))
    left_track.append(mido.Message('note_on', note = 60, time = 0))
#     left_track.append(mido.Message('note_off', note = 81, time = 256))
#     left_track.append(mido.Message('note_off', note = 60, time = 10))

    
#     mid.tracks.append(right_track)
    mid.tracks.append(left_track)

    mid.save('align2.mid')

In [20]:
create_midi()

In [21]:
# pip install rouge_metric
from rouge_metric import PyRouge

In [22]:
all_songs_notes = convert_list_of_messages_to_notes(create_list_of_messages("./midis", "Piano left"))

In [23]:
#generated_hand == 
    # generated_left_hand 
    # or generated_right_hand 
# all_songs_notes ==
    # convert_list_of_messages_to_notes(create_list_of_messages("./midis", "Piano left"))
    # convert_list_of_messages_to_notes(create_list_of_messages("./midis", "Piano right"))
def get_rouge_metric(generated_hand, all_songs_notes):
    r_list = []
    f_list = []
    p_list = []
    for song in all_songs_notes:
        hypotheses = list(map(str, generated_hand))
        references = list(map(str, song))

        if len(hypotheses) > len(references):
            hypotheses = hypotheses[0:len(references)]
        elif len(hypotheses) < len(references):
            references = references[0:len(hypotheses)]
        # Evaluate on tokenized documents
        rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                        rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
        scores = rouge.evaluate_tokenized(hypotheses, references)
        if 'rouge-1' in scores:
            r_list.append(scores['rouge-1']['r'])
            p_list.append(scores['rouge-1']['p'])
            f_list.append(scores['rouge-1']['f'])
            
    return max(r_list), max(p_list), max(f_list)

In [24]:
get_rouge_metric(generated_left_hand, all_songs_notes)


(0.5645161290322581, 0.28225806451612906, 0.3763440860215054)